### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [3]:
ALL_IN_ONE_WORKER = True

### Start with our Message class

In [4]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [5]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

### Let's reintroduce a tool

In [6]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [7]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [8]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [9]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [10]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [12]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several pros of using AutoGen for your AI Agent project:

1. **Multi-Agent Collaboration**: AutoGen excels in scenarios that require cooperation between multiple AI agents, making it ideal for complex tasks that involve decision-making or back-and-forth interactions.

2. **Modular Design**: The framework's modularity allows for easy maintenance and scalability, accommodating both simple and complex multi-agent scenarios.

3. **Customizability**: AutoGen offers advanced customization options, enabling you to define specific roles, capabilities, and communication protocols for each agent, tailoring them to your project needs.

4. **User-Friendly Framework**: It is designed to be accessible, accelerating development and research in agentic AI while providing out-of-the-box agent templates for quick setups.

5. **Flexible Tool Orchestration**: AutoGen supports structured management of agents, including tool orchestration and memory management, enhancing their overall efficacy.

6. **Human-in-the-Loop Support**: The framework allows for implementing human oversight in workflows, improving the reliability and safety of AI outputs.

In summary, AutoGen is advantageous for its collaborative capabilities, modularity, customization, user-friendliness, flexible orchestration, and human-in-the-loop features, making it a strong contender for AI agent projects.

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI Agent project:

1. **Resource Intensiveness**: AutoGen can be significantly resource-intensive, requiring more computational power, which may lead to higher operational costs, especially in large-scale applications.

2. **Complex Setup**: It often demands more initial setup and configuration than simpler frameworks, which can lead to higher upfront development time and resource allocation.

3. **Steep Learning Curve**: New developers may encounter a steep learning curve, needing substantial time to become proficient in utilizing AutoGen effectively.

4. **Less Structure**: Some developers may find AutoGen to be less structured compared to other agent frameworks, which could complicate the implementation process and lead to inconsistencies in development.

5. **Limited Customization Options**: AutoGen may have restrictions on how customizable and integrable it is with other systems, which can hamper flexibility in meeting specific project needs.

6. **Monitoring Overhead**: The need to constantly monitor the use of credits or resources in AutoGen can be burdensome and may lead to inefficiencies and unexpected costs.

7. **Limited AI Capabilities**: It may not be ideally suited for complex AI-driven automation tasks, which may limit its applicability in more advanced projects.

These factors should be weighed carefully when considering AutoGen for your AI Agent project. 

TERMINATE



## Decision:

Based on the research provided by the team, I have decided to move forward with AutoGen for the AI Agent project. 

The rationale behind this decision hinges on the significant advantages that AutoGen offers, particularly its strong support for multi-agent collaboration and customizable frameworks. These features align well with the project's need for complex interactions and tailored solutions. Although the cons, such as resource intensiveness and a complex setup, present valid concerns, the benefits of flexibility, maintenance ease, and human oversight in decision-making outweigh the drawbacks. 

Therefore, AutoGen represents a promising solution that can enhance the overall efficacy and success of the project. 

TERMINATE

In [13]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [14]:
await host.stop()